Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License. 

# Setup environment for module CLI and SDK

In this tutorial you will learn how to work with Azure ML module:

1. Setup environment - install module CLI and module/pipeline SDK
2. Prepare workspace
3. Next steps:
   - Registering and managing modules using module CLI
   - Use module/pipeline SDK to create a pipeline with registered modules.


## Prerequisites

* Install Azure CLI, please follow [the Azure CLI installation instructions](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest) to install.

## Setup environment
* Install Azure CLI AML extension which includes the `az ml module` command group
* Install Azure ML SDK including the APIs to work with _module_ and _pipeline_

Install the latest version of module CLI.

In [1]:
# Uninstall azure-cli-ml (the `az ml` commands)
!az extension remove -n azure-cli-ml 

# Specify the latest version here
%env CLI_SDK_VERSION=15324996

# Install local version of azure-cli-ml (which includes `az ml module` commands)
!az extension add --source https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/$CLI_SDK_VERSION/azure_cli_ml-0.1.0.$CLI_SDK_VERSION-py3-none-any.whl --pip-extra-index-urls https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/$CLI_SDK_VERSION --yes 

env: CLI_SDK_VERSION=15324996
 - Downloading .. - Validating .. - Installing ..

In [2]:
# Verify the version of the azure-cli-ml extension
!az extension list -o table

Experimental    ExtensionType    Name          Path                                            Preview    Version
--------------  ---------------  ------------  ----------------------------------------------  ---------  --------------
False           whl              azure-cli-ml  /home/zhizhu/.azure/cliextensions/azure-cli-ml  False      0.1.0.15324996


Install the latest version of module SDK.

In [3]:
%config IPCompleter.greedy=True
!pip install azureml-pipeline-wrapper[notebook]==0.1.0.$CLI_SDK_VERSION --extra-index-url https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/$CLI_SDK_VERSION --user --upgrade


Looking in indexes: https://pypi.org/simple, https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/15324996
Requirement already up-to-date: azureml-pipeline-wrapper==0.1.0.15324996 in /home/zhizhu/.local/lib/python3.7/site-packages (0.1.0.15324996)


Restart the kernel to make sure the newly installed package applied successfully.

## Specify information of the workspace 


In [4]:
# Update the following information with your environment

SUBSCRIPTION_ID = 'e9b2ec51-5c94-4fa8-809a-dc1e695e4896'
WORKSPACE_NAME = 'sample-ws'
RESOURCE_GROUP_NAME = 'sample-ws-rg'

## Login and set the default subscription

Login with the azure CLI and optionally set the default subscription.

In [5]:
!az login -o table

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BUAEXPBGA to authenticate.
CloudName    HomeTenantId                          IsDefault    Name                                                            State    TenantId
-----------  ------------------------------------  -----------  --------------------------------------------------------------  -------  ------------------------------------
AzureCloud   72f988bf-86f1-41af-91ab-2d7cd011db47  False        Deep Learning Group                                             Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   72f988bf-86f1-41af-91ab-2d7cd011db47  False        Aml.DataScience                                                 Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   72f988bf-86f1-41af-91ab-2d7cd011db47  False        Epic-Edge-ES-MergeResolver                                      Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   72f988bf-86f1-41af

Set default subscription using the following command if needed.

In [6]:
!az account set -s $SUBSCRIPTION_ID

## Prepare workspace

Prepare a workspace to register modules and create pipelines.
* Create a new workspace if not already have one.
* Create default data types for the newly created workspace.
* Set the workspace as default.


If the workspace or resource group does not already exist, use the following command to create.

In [7]:
!az group create -g $RESOURCE_GROUP_NAME --location eastus
!az ml workspace create -w $WORKSPACE_NAME -g $RESOURCE_GROUP_NAME --sku=enterprise

{
  "id": "/subscriptions/e9b2ec51-5c94-4fa8-809a-dc1e695e4896/resourceGroups/sample-ws-rg",
  "location": "eastus",
  "managedBy": null,
  "name": "sample-ws-rg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}
Deploying KeyVault with name samplewskeyvault3bb474d3.
Deploying AppInsights with name samplewsinsights69842907.
Deployed AppInsights with name samplewsinsights69842907. Took 3.15 seconds.
Deploying StorageAccount with name samplewsstorage38faceb8c.
Deployed KeyVault with name samplewskeyvault3bb474d3. Took 19.14 seconds.
Deploying Workspace with name sample-ws.
Deployed StorageAccount with name samplewsstorage38faceb8c. Took 22.53 seconds.
Deployed Workspace with name sample-ws. Took 131.99 seconds.
{
  "applicationInsights": "/subscriptions/e9b2ec51-5c94-4fa8-809a-dc1e695e4896/resourcegroups/sample-ws-rg/providers/microsoft.insights/components/samplewsinsights69842907",
  "creationTime": "2020-06-09T

Use the following command to set the default workspace. After setting default workspace:
* The workspace could be retrieved using `Workspace.from_config()` from Python code.
* It is not required to specify workspace information for each `az ml` command, the workspace information could be retrieved automatically from the config.

In [8]:
!az ml folder attach -w $WORKSPACE_NAME -g $RESOURCE_GROUP_NAME

{
  "Experiment name": "notebooks",
  "Project path": "/home/zhizhu/ModuleDocs/notebooks",
  "Resource group": "sample-ws-rg",
  "Subscription id": "e9b2ec51-5c94-4fa8-809a-dc1e695e4896",
  "Workspace name": "sample-ws"
}


For newly created workspaces, the default DataTypes (`AnyFile`, `AnyDirectory`) should be registered before module registration.

Use the following code to register defalt data types:

In [9]:
from azureml.core import Workspace
from azureml.pipeline.core.graph import DataType

ws = Workspace.from_config()
# call list_data_type will trigger the registration of default datatypes such as AnyFile, AnyDirectory
DataType.list_data_types(ws)

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


## Next steps

Now you are prepared for the module CLI and SDK. You could check the following instructions for further tasks:
* [Operating modules using the module CLI](azureml-module-cli-walk-through.ipynb)
* [Creating a pipeline using module SDK](azureml-module-create-pipeline.ipynb)